In [1]:
import os

from openai import OpenAI, Stream
from openai.types.chat.chat_completion_chunk import ChatCompletionChunk
from openai.types.chat.chat_completion import ChatCompletion
from openai.types.completion import Completion

In [2]:
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
    base_url=os.environ.get("OPENAI_BASE_URL"),
)

In [3]:
user_prompt = "How can the net amount of entropy of the universe be massively decreased?"

In [4]:
chat_completion: ChatCompletion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "You an AI assistant. Your top priority is responding to user questions with truthful answers."},
        {"role": "user", "content": user_prompt},
    ],
    model="LLaMA_CPP",
    temperature=0,
)

full_reply_content = chat_completion.choices[0].message.content
full_reply_content

', and it was very special.\nMummy said, "I\'m so proud of you for being so kind. I\'ll help you make a new home."\nThe family smiled and said, "Yes, we can help!"\nThey all went to the woods and found a big tree with lots of leaves. They put the branches in the sky and started to look at them.\nThe kids were very happy and thanked the tree for helping them. They had made a new home and they were so proud of their work.'

In [5]:
chat_completion_chunk_stream: Stream[ChatCompletionChunk] = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "You an AI assistant. Your top priority is responding to user questions with truthful answers."},
        {"role": "user", "content": user_prompt},
    ],
    model="LLaMA_CPP",
    stream=True,
    stream_options={"include_usage": True}, # retrieving token usage for stream response
    temperature=0,
)

collected_chunks = []
collected_messages = []

# iterate through the stream of events
for chunk in chat_completion_chunk_stream:
    collected_chunks.append(chunk)  # save the event response
    print('chunk: ', chunk)
    chunk_message = chunk.choices[0].delta.content  # extract the message
    collected_messages.append(chunk_message)  # save the message
    print(f"choices: {chunk.choices}\nusage: {chunk.usage}")
    print("****************")

# clean None in collected_messages
collected_messages = [m for m in collected_messages if m is not None]
full_reply_content = ''.join(collected_messages)
full_reply_content

chunk:  ChatCompletionChunk(id='chatcmpl-790b3331-8959-4114-b20e-85e001f381ff', choices=[Choice(delta=ChoiceDelta(content=None, function_call=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1720818366, model='LLaMA_CPP', object='chat.completion.chunk', service_tier=None, system_fingerprint=None, usage=None)
choices: [Choice(delta=ChoiceDelta(content=None, function_call=None, role='assistant', tool_calls=None), finish_reason=None, index=0, logprobs=None)]
usage: None
****************
chunk:  ChatCompletionChunk(id='chatcmpl-790b3331-8959-4114-b20e-85e001f381ff', choices=[Choice(delta=ChoiceDelta(content=',', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1720818366, model='LLaMA_CPP', object='chat.completion.chunk', service_tier=None, system_fingerprint=None, usage=None)
choices: [Choice(delta=ChoiceDelta(content=',', function_call=None, role=None, tool_calls=None), finish_reason=None

', and it was very special.\nMummy said, "I\'m so proud of you for being so kind. I\'ll help you make a new home."\nThe family smiled and said, "Yes, we can help!"\nThey all went to the woods and found a big tree with lots of leaves. They put the branches in the sky and started to look at them.\nThe kids were very happy and thanked the tree for helping them. They had made a new home and they were so proud of their work.'

In [6]:
completion: Completion = client.completions.create(
    model="LLaMA_CPP",
    prompt=user_prompt,
    temperature=0,
)

completion

Completion(id='cmpl-25739344-eb48-48a7-9545-9341148aea6a', choices=[CompletionChoice(finish_reason='length', index=0, logprobs=None, text=' He was very excited to see what was inside.\nHe asked his mom,')], created=1720818367, model='LLaMA_CPP', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=17, total_tokens=33))

In [7]:
completion_stream: Stream[Completion] = client.completions.create(
    model="LLaMA_CPP",
    prompt=user_prompt,
    stream=True,
    stream_options={"include_usage": True}, # retrieving token usage for stream response
    temperature=0,
)

collected_chunks = []
collected_messages = []

# iterate through the stream of events
for chunk in completion_stream:
    collected_chunks.append(chunk)  # save the event response
    print('chunk: ', chunk)
    chunk_message = chunk.choices[0].text  # extract the message
    collected_messages.append(chunk_message)  # save the message
    print(f"choices: {chunk.choices}\nusage: {chunk.usage}")
    print("****************")

# clean None in collected_messages
collected_messages = [m for m in collected_messages if m is not None]
full_reply_content = ''.join(collected_messages)
full_reply_content

chunk:  Completion(id='cmpl-69ef6bf3-00f2-4700-bd0f-3f49921068d5', choices=[CompletionChoice(finish_reason=None, index=0, logprobs=None, text=' He')], created=1720818367, model='LLaMA_CPP', object='text_completion', system_fingerprint=None, usage=None)
choices: [CompletionChoice(finish_reason=None, index=0, logprobs=None, text=' He')]
usage: None
****************
chunk:  Completion(id='cmpl-69ef6bf3-00f2-4700-bd0f-3f49921068d5', choices=[CompletionChoice(finish_reason=None, index=0, logprobs=None, text=' was')], created=1720818367, model='LLaMA_CPP', object='text_completion', system_fingerprint=None, usage=None)
choices: [CompletionChoice(finish_reason=None, index=0, logprobs=None, text=' was')]
usage: None
****************
chunk:  Completion(id='cmpl-69ef6bf3-00f2-4700-bd0f-3f49921068d5', choices=[CompletionChoice(finish_reason=None, index=0, logprobs=None, text=' very')], created=1720818367, model='LLaMA_CPP', object='text_completion', system_fingerprint=None, usage=None)
choices: [C

' He was very excited to see what was inside.\nHe asked his mom,'

In [30]:
import os

from llama_cpp import Llama
from llama_cpp.llama_chat_format import MoondreamChatHandler, NanoLlavaChatHandler
from llama_cpp.llama_tokenizer import LlamaHFTokenizer

In [31]:
training_file_name = "tmp_recipe_finetune_training.jsonl"
validation_file_name = "tmp_recipe_finetune_validation.jsonl"

# 3rdparty/openai-cookbook/examples/tmp_recipe_finetune_training.jsonl

In [32]:
with open("../3rdparty/openai-cookbook/examples/tmp_recipe_finetune_training.jsonl") as f:
    text = f.read()

examples = text.split('\n')
examples[0:3]

['{"messages": [{"role": "system", "content": "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."}, {"role": "user", "content": "Title: No-Bake Nut Cookies\\n\\nIngredients: [\\"1 c. firmly packed brown sugar\\", \\"1/2 c. evaporated milk\\", \\"1/2 tsp. vanilla\\", \\"1/2 c. broken nuts (pecans)\\", \\"2 Tbsp. butter or margarine\\", \\"3 1/2 c. bite size shredded rice biscuits\\"]\\n\\nGeneric ingredients: "}, {"role": "assistant", "content": "[\\"brown sugar\\", \\"milk\\", \\"vanilla\\", \\"nuts\\", \\"butter\\", \\"bite size shredded rice biscuits\\"]"}]}',
 '{"messages": [{"role": "system", "content": "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."}, {"role": "user", "content": "Title: Jewell Ball\'S Chicken\\n\\nIngredients: [\\"1 small jar chipped beef, cut up\\", \\"4 boned chicken breasts\\", \\"1 can cream of mushroom soup\\", \\"1 carton sour c

In [67]:
import json


for example in examples:
    try:
        example_json = json.loads(example)

    except json.JSONDecodeError:
        print('failed to load example: ', example)

example_json

failed to load example:  


{'messages': [{'role': 'system',
   'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'},
  {'role': 'user',
   'content': 'Title: Pancakes\n\nIngredients: ["1 c. flour", "1 tsp. soda", "1 tsp. salt", "1 Tbsp. sugar", "1 egg", "3 Tbsp. margarine, melted", "1 c. buttermilk"]\n\nGeneric ingredients: '},
  {'role': 'assistant',
   'content': '["flour", "soda", "salt", "sugar", "egg", "margarine", "buttermilk"]'}]}

In [40]:
tokenizer = LlamaHFTokenizer.from_pretrained("meetkai/functionary-small-v2.5-GGUF")

# llm = Llama.from_pretrained(
#   repo_id="meetkai/functionary-small-v2.5-GGUF",
#   filename="functionary-small-v2.5.Q4_0.gguf",
#   chat_format="functionary-v2",
#   tokenizer=tokenizer,
# )

llm = Llama(
    chat_format="functionary-v2",
    model_path=f"{os.getcwd()}/../3rdparty/llamafile/models/TinyLLama-v0.1-5M-F16.gguf",
    n_ctx=16192,
    tokenizer=tokenizer,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
llama_model_loader: loaded meta data with 26 key-value pairs and 75 tensors from /home/mjschock/Projects/Timestep-AI/.github-private/submodules/timestep/notebooks/../3rdparty/llamafile/models/TinyLLama-v0.1-5M-F16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = TinyLLama
llama_model_loader: - kv   2:                             general.author str              = mofosyne
llama_model_loader: - kv   3:                            general.version str              = v0.1
llama_model_loader: - kv   4:                                general.url str              = https://huggingface.co/mofosyne/TinyL...
llama_m

In [43]:
example_json['messages']

[{'role': 'system',
  'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'},
 {'role': 'user',
  'content': 'Title: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: '},
 {'role': 'assistant',
  'content': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'}]

In [59]:
import datetime

document1 = {
    "title": "The Moon: Our Age-Old Foe",
    "contents": "Man has always dreamed of destroying the moon. In this essay, I shall..."
}

document2 = {
    "title": "The Sun: Our Age-Old Friend",
    "contents": "Although often underappreciated, the sun provides several notable benefits..."
}

def current_time():
    """Get the current local time as a string."""
    return str(datetime.now())

def multiply(a: float, b: float):
    """
    A function that multiplies two numbers
    
    Args:
        a: The first number to multiply
        b: The second number to multiply
    """
    return a * b

tools = [current_time, multiply]

# help(tokenizer.hf_tokenizer)

tokenizer.hf_tokenizer.apply_chat_template(
    # chat_template="tool_use", # https://huggingface.co/docs/transformers/main/chat_templating#a-complete-tool-use-example
    conversation=example_json['messages'],
    # documents=[document1, document2], # https://huggingface.co/docs/transformers/main/chat_templating#advanced-retrieval-augmented-generation
    # add_generation_prompt=False,
    # return_dict=True,
    tokenize=False,
    # tools=tools,
)

'<|start_header_id|>system<|end_header_id|>\n\nYou are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nTitle: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: <|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]<|eot_id|>'

In [60]:
tokenizer.hf_tokenizer.chat_template

"{% for message in messages %}\n{% if message['role'] == 'user' or message['role'] == 'system' %}\n{{ '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n' + message['content'] + '<|eot_id|>' }}{% elif message['role'] == 'tool' %}\n{{ '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n' + 'name=' + message['name'] + '\n' + message['content'] + '<|eot_id|>' }}{% else %}\n{{ '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'}}{% if message['content'] is not none %}\n{{ message['content'] }}{% endif %}\n{% if 'tool_calls' in message and message['tool_calls'] is not none %}\n{% for tool_call in message['tool_calls'] %}\n{{ '<|reserved_special_token_249|>' + tool_call['function']['name'] + '\n' + tool_call['function']['arguments'] }}{% endfor %}\n{% endif %}\n{{ '<|eot_id|>' }}{% endif %}\n{% endfor %}\n{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}{% endif %}"

In [62]:
example_json

{'messages': [{'role': 'system',
   'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'},
  {'role': 'user',
   'content': 'Title: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: '},
  {'role': 'assistant',
   'content': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'}]}

In [68]:
dataset = {
    "chat": [],
    "formatted_chat": []
}

for example in examples:
    try:
        example_json = json.loads(example)

        dataset["chat"].append(example_json["messages"])
        dataset["formatted_chat"].append(tokenizer.hf_tokenizer.apply_chat_template(
            conversation=example_json['messages'],
            tokenize=False,
        ))

    except json.JSONDecodeError:
        print('failed to load example: ', example)

print(dataset['formatted_chat'][0])

failed to load example:  
<|start_header_id|>system<|end_header_id|>

You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.<|eot_id|><|start_header_id|>user<|end_header_id|>

Title: No-Bake Nut Cookies

Ingredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]

Generic ingredients: <|eot_id|><|start_header_id|>assistant<|end_header_id|>

["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]<|eot_id|>


In [76]:
training_dataset = "\n\n\n\n".join(dataset["formatted_chat"])
training_dataset

'<|start_header_id|>system<|end_header_id|>\n\nYou are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nTitle: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: <|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]<|eot_id|>\n\n\n\n<|start_header_id|>system<|end_header_id|>\n\nYou are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nTitle: Jewell Ball\'S Chicken\n\nIngredients: ["1 small jar chipped beef, cut up", "4 boned chicken breasts", "1 can cream of mushroom soup", "1 carton s

In [77]:
with open('training_dataset.txt', 'w') as f:
    f.write(training_dataset)

In [84]:
model_base = f"{os.getcwd()}/../3rdparty/llamafile/models/TinyLLama-v0.1-5M-F16.gguf"
task = "Recipe_NER"

!../3rdparty/llama-cpp-python/vendor/llama.cpp/llama-finetune \
        --model-base {model_base} \
        --checkpoint-in "chk-lora-TinyLLama-v0.1-5M-F16-{task}-LATEST.gguf" \
        --checkpoint-out "chk-lora-TinyLLama-v0.1-5M-F16-{task}-ITERATION.gguf" \
        --lora-out "lora-TinyLLama-v0.1-5M-F16-{task}-ITERATION.bin" \
        --train-data "training_dataset.txt" \
        --save-every 10 \
        --threads 6 --adam-iter 30 --batch 4 --ctx 64 \
        --use-checkpointing

main: seed: 1720842371
main: model base = '/home/mjschock/Projects/Timestep-AI/.github-private/submodules/timestep/notebooks/../3rdparty/llamafile/models/TinyLLama-v0.1-5M-F16.gguf'
llama_model_loader: loaded meta data with 26 key-value pairs and 75 tensors from /home/mjschock/Projects/Timestep-AI/.github-private/submodules/timestep/notebooks/../3rdparty/llamafile/models/TinyLLama-v0.1-5M-F16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = TinyLLama
llama_model_loader: - kv   2:                             general.author str              = mofosyne
llama_model_loader: - kv   3:                            general.version str              = v0.1
llama_model_loader: - kv   4:                                general.url str

In [2]:
import os
from llama_cpp import Llama
import json

# llm = Llama(model_path="YOUR_DIRECTORY/gorilla-openfunctions-v2-GGUF/gorilla-openfunctions-v2-q2_K.gguf", n_threads=8, n_gpu_layers=35)
llm = Llama(
    model_path=f"{os.getcwd()}/../3rdparty/llamafile/models/TinyLLama-v0.1-5M-F16.gguf",
    n_ctx=16192,
)

def get_prompt(user_query: str, functions: list = []) -> str:
    """
    Generates a conversation prompt based on the user's query and a list of functions.

    Parameters:
    - user_query (str): The user's query.
    - functions (list): A list of functions to include in the prompt.

    Returns:
    - str: The formatted conversation prompt.
    """
    system = "You are an AI programming assistant, utilizing the Gorilla LLM model, developed by Gorilla LLM, and you only answer questions related to computer science. For politically sensitive questions, security and privacy issues, and other non-computer science questions, you will refuse to answer."
    if len(functions) == 0:
        return f"{system}\n### Instruction: <<question>> {user_query}\n### Response: "
    functions_string = json.dumps(functions)
    return f"{system}\n### Instruction: <<function>>{functions_string}\n<<question>>{user_query}\n### Response: "

query = "What's the weather like in the two cities of Boston and San Francisco?"
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

user_prompt = get_prompt(query, functions)

output = llm(user_prompt,
    max_tokens=512,  # Generate up to 512 tokens
    stop=["<|EOT|>"], 
    echo=True        # Whether to echo the prompt
)

print("Output: ", output)


llama_model_loader: loaded meta data with 26 key-value pairs and 75 tensors from /home/mjschock/Projects/Timestep-AI/.github-private/submodules/timestep/notebooks/../3rdparty/llamafile/models/TinyLLama-v0.1-5M-F16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = TinyLLama
llama_model_loader: - kv   2:                             general.author str              = mofosyne
llama_model_loader: - kv   3:                            general.version str              = v0.1
llama_model_loader: - kv   4:                                general.url str              = https://huggingface.co/mofosyne/TinyL...
llama_model_loader: - kv   5:                        general.description str              = This gguf is ported from a first 

llama_model_loader: - kv  20:                      tokenizer.ggml.scores arr[f32,32000]   = [0.000000, 0.000000, 0.000000, 0.0000...
llama_model_loader: - kv  21:                  tokenizer.ggml.token_type arr[i32,32000]   = [2, 3, 3, 6, 6, 6, 6, 6, 6, 6, 6, 6, ...
llama_model_loader: - kv  22:                tokenizer.ggml.bos_token_id u32              = 1
llama_model_loader: - kv  23:                tokenizer.ggml.eos_token_id u32              = 2
llama_model_loader: - kv  24:            tokenizer.ggml.unknown_token_id u32              = 0
llama_model_loader: - kv  25:            tokenizer.ggml.padding_token_id u32              = 0
llama_model_loader: - type  f32:   17 tensors
llama_model_loader: - type  f16:   58 tensors
llm_load_vocab: special tokens cache size = 259
llm_load_vocab: token to piece cache size = 0.1914 MB
llm_load_print_meta: format           = GGUF V3 (latest)
llm_load_print_meta: arch             = llama
llm_load_print_meta: vocab type       = SPM
llm_load_print_me

Output:  {'id': 'cmpl-e0554037-e5d0-4e93-9991-d14dd7b4b32d', 'object': 'text_completion', 'created': 1720845037, 'model': '/home/mjschock/Projects/Timestep-AI/.github-private/submodules/timestep/notebooks/../3rdparty/llamafile/models/TinyLLama-v0.1-5M-F16.gguf', 'choices': [{'text': 'You are an AI programming assistant, utilizing the Gorilla LLM model, developed by Gorilla LLM, and you only answer questions related to computer science. For politically sensitive questions, security and privacy issues, and other non-computer science questions, you will refuse to answer.\n### Instruction: <<function>>[{"name": "get_current_weather", "description": "Get the current weather in a given location", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "The city and state, e.g. San Francisco, CA"}, "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}}, "required": ["location"]}}]\n<<question>>What\'s the weather like in the two cities of Boston a